In [1]:
#Reference :
#RandomForest
#https://github.com/apache/spark/blob/v2.2.0/mllib/src/main/scala/org/apache/spark/ml/classification/RandomForestClassifier.scala#L120
#One hot encoding
#https://towardsdatascience.com/machine-learning-with-pyspark-and-mllib-solving-a-binary-classification-problem-96396065d2aa
#LR from scratch
#https://medium.com/@martinpella/logistic-regression-from-scratch-in-python-124c5636b8ac
#https://www.programcreek.com/python/example/106727/pyspark.ml.feature.StringIndexer

In [2]:
!pip install --upgrade pip
!pip install --upgrade pandas
!pip install --upgrade google-api-python-client
!pip install --upgrade seaborn
!pip install --upgrade networkx
!pip install --upgrade matplotlib
!pip install --upgrade pyspark
!pip install --upgrade pyspark_dist_explore


    100% |████████████████████████████████| 1.3MB 17.1MB/s ta 0:00:01
pyspark 2.3.1 requires py4j==0.10.7, which is not installed.
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1
Requirement already up-to-date: pandas in /opt/conda/lib/python3.6/site-packages (0.23.4)
    100% |████████████████████████████████| 61kB 3.9MB/s ta 0:00:011
    100% |████████████████████████████████| 71kB 9.1MB/s ta 0:00:011
    100% |████████████████████████████████| 215kB 38.5MB/s ta 0:00:01
    100% |████████████████████████████████| 1.7MB 19.4MB/s ta 0:00:01
  Running setup.py bdist_wheel for networkx ... done
  Stored in directory: /root/.cache/pip/wheels/68/f8/29/b53346a112a07d30a5a84d53f19aeadaa1a474897c0423af91
Successfully built networkx
    100% |████████████████████████████████| 12.9MB 3.1MB/s eta 0:00:01
  Found existing installation: matplotlib 3.0.1
    Uninstalling matplotlib-3.0.1:
      Successfully uninstalled matplotlib-3.0.

  Running setup.py bdist_wheel for pyspark ... done
  Stored in directory: /root/.cache/pip/wheels/cd/54/c2/abfcc942eddeaa7101228ebd6127a30dbdf903c72db4235b23
Successfully built pyspark
  Found existing installation: pyspark 2.3.1
    Can't uninstall 'pyspark'. No files were found to uninstall.


In [71]:
import re
import ast
import time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
import os
from IPython.display import display, HTML, display_html #usefull to display wide tables
from pyspark_dist_explore import Histogram, hist, distplot, pandas_histogram
from pyspark.sql import functions as F, types
import datetime
from pyspark.sql.functions import col, row_number, concat, lit
from pyspark.sql.window import Window
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import trim
import pickle
from pyspark.sql.functions import (explode, col)
from pyspark.mllib.linalg import SparseVector
from pyspark.sql.functions import udf, split
from pyspark.sql.types import ArrayType, StructType, StructField, LongType, StringType

from pyspark.sql.functions import udf
#from pyspark.mllib.linalg import VectorUDT
from pyspark.ml.linalg import VectorUDT


In [31]:
%matplotlib inline


In [32]:
spark

In [5]:
#spark.conf.set("spark.executor.memory", '19g')


In [33]:
sc = spark.sparkContext

In [34]:
sc.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.dynamicAllocation.minExecutors', '1'),
 ('spark.driver.maxResultSize', '13312m'),
 ('spark.eventLog.dir', 'hdfs://cluster-w261-m/user/spark/eventlog'),
 ('spark.ui.proxyBase', '/proxy/application_1544416808440_0001'),
 ('spark.yarn.am.memory', '640m'),
 ('spark.driver.port', '41669'),
 ('spark.driver.appUIAddress',
  'http://cluster-w261-m.c.w266-203603.internal:4040'),
 ('spark.executor.instances', '2'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  'cluster-w261-m'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.yarn.historyServer.address', 'cluster-w261-m:18080'),
 ('spark.submit.deployMode', 'client'),
 ('spark.executor.memory', '40g'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spark.executor.cores', '8'),
 ('spark.history.fs.logDirectory',
  'hdfs://cluster-w261-m/user/spark/eventlog'),
 ('spark.driver.memory', '26624m'),
 ('spark.shu

In [66]:
# spark.conf.set('spark.executor.cores', '5')


In [8]:
#https://blog.cloudera.com/blog/2015/03/how-to-tune-your-apache-spark-jobs-part-2/

#The number of cores can be specified with the --executor-cores flag when invoking 
#spark-submit, spark-shell, and pyspark from the command line, 
#or by setting the spark.executor.cores property in the spark-defaults.conf file 
#or on a SparkConf object.

#The cores property controls the number of concurrent tasks an executor can run. 
#--executor-cores 5 means that each executor can run a maximum of five tasks at the same time.

#The heap size can be controlled with the --executor-memory flag 
#or the spark.executor.memory property
#The memory property impacts the amount of data Spark can cache, 
#as well as the maximum sizes of the shuffle data structures used for grouping, aggregations, and joins.

#The --num-executors command-line flag or spark.executor.instances configuration property 
#control the number of executors requested. Starting in CDH 5.4/Spark 1.3, 
#you will be able to avoid setting this property by turning on dynamic allocation with the spark.dynamicAllocation.enabled property. Dynamic allocation enables a Spark application to request executors 
#when there is a backlog of pending tasks and free up executors when idle.


In [35]:
def f_save_to_bucket(inObject, inFileName):
    !mkdir -p ./data
    filePath=os.path.join('data',inFileName+'.pkl')
    with open(filePath, 'wb') as fp:
        pickle.dump(inObject, fp)
    !hadoop fs -put -f {filePath} gs://bucket-w261-final/{filePath}
    !hadoop fs -ls gs://bucket-w261-final/{filePath}


In [24]:
# def f_load_from_bucket(inObject, inFileName):
#     hadoop fs -get 
#     outObject = pickle.load( open( inFileName, "rb" ) )
#     !mkdir -p ./data
#     filePath=os.path.join('data',inFileName+'.pkl')
#     with open(filePath, 'wb') as fp:
#         pickle.dump(inObject, fp)
#     !hadoop fs -put -f {filePath} gs://bucket-w261-final/{filePath}
#     !hadoop fs -ls gs://bucket-w261-final/{filePath}


In [7]:
def f_calc_stats(data, column):
        return data.agg(F.avg(data[column]), F.min(data[column]), F.max(data[column]), \
                        F.stddev_pop(data[column]),F.var_pop(data[column]),F.skewness(data[column]) \
                       ).collect()

In [8]:
def f_check_null(data, column):
    return data.filter( (data[column] =="") |F.isnull(data[column])|F.isnan(data[column])
                      ).count()

In [15]:
#Checking cardinality
def f_display_stats_categ(df, inColList):
    dict1={}
    for col in inColList:
        cardinal_cnt = df.select([col]).distinct().count()
        dict1[col]={"Count_Unique_Vals":cardinal_cnt}
        dict1[col]['Count_Empty_String'] = str(df.filter(df[col] == "").count())

    display(HTML(pd.DataFrame(dict1).T.to_html( )))

In [41]:
def f_display_stats_int(data):
    dict1={}
    countTotal = data.count()
    for colname in [item[0] for item in data.dtypes if item[1].startswith('int')]:
        list1=f_calc_stats(data,colname)
        mean_val, min_val,max_val,stddev,var, skewness =list1[0]
        count_nulls = f_check_null(data,colname)
        dict1[colname]={}
        dict1[colname]['mean'] = str(round(mean_val,2))
        dict1[colname]['min'] = str(min_val)
        dict1[colname]['max'] = str(max_val)
        dict1[colname]['stddev'] = str(round(stddev,2))
        dict1[colname]['var'] = str(round(var,2))
        dict1[colname]['skewness'] = str(round(skewness,2))
        dict1[colname]['nulls_nans'] = str(count_nulls)
        dict1[colname]['pct_nulls_nans'] = str(round(float(count_nulls/countTotal*100),2))
        dict1[colname]['count_empty_string'] = str(data.filter(data[colname] == "").count())
        dict1[colname]['count_unique_values'] = str(data.select([colname]).distinct().count())
        dict1[colname]['count'] = 'N/A'
    dict1['TOTAL']={}
    dict1['TOTAL']['count'] = str(data.count())
   #Transposing dataframe to keep column names as rows
    display(HTML(pd.DataFrame(dict1).T.to_html( )))

In [17]:
#Create dataframe from RDD
def f_covert_to_df(rdd, col_list):
    return rdd.map(lambda x: x.split('\t')).toDF(col_list)

In [18]:
#Convert integer columns to IntegerType from String
def f_cast_str_to_int(df, integer_col_list):
    for col in integer_col_list:
        df = df.withColumn(col, df[col].cast(types.IntegerType()))
    return df

In [19]:
#Removing rows where at least one column has empty string
def f_remove_empty_string(df, categ_col_list):
    for categ_col in categ_col_list:
        df = df.filter(df[categ_col] != "")
        #df = df.filter(col(categ_col) != "")
    return df

In [89]:
#Converting categorical variables to one-hot encoding
def f_onehot_encoding(df,label_col_list, int_col_list, categ_col_list):
    stages=[]
    for categoricalCol in categ_col_list:
        stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
#         encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
        encoder = OneHotEncoderEstimator(handleInvalid='keep', inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
        stages += [stringIndexer, encoder]
        
    #Skip for test data since test data does not have label
    if label_col_list is not None:
        labelColStr=''.join(label_col_list)
        label_stringIdx = StringIndexer(inputCol = labelColStr, outputCol = 'label')
        stages += [label_stringIdx]

    assemblerInputs = [c + "classVec" for c in categ_col_list] + int_col_list
    assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
    stages += [assembler]

    cols = df.columns

#     partialPipeline = Pipeline(stages = stages)
    partialPipeline = Pipeline().setStages(stages)
    pipelineModel = partialPipeline.fit(df)

    preppedDataDF = pipelineModel.transform(df)

#     if label_col_list is not None:
#          selectedCols = ['label', 'features'] + cols
#     else:
#         selectedCols = ['features'] + cols
        
#     preppedDataDF = preppedDataDF.select(selectedCols)

    return preppedDataDF

In [106]:
# trainRDD = sc.textFile('gs://bucket-w261-final/data/train.txt',50)
# trainRDD.cache()

In [107]:
# testRDD = sc.textFile('gs://bucket-w261-final/data/test.txt',50)
# testRDD.cache()

In [23]:
# labelColList=["clicked_0_1"]
# intColList=["int_1", "int_2", "int_3", "int_4", "int_5", "int_6", "int_7", "int_8", "int_9", "int_10", "int_11", "int_12", "int_13"]
# categColList=["categ_1", "categ_2", "categ_3", "categ_4", "categ_5", "categ_6", "categ_7", "categ_8", "categ_9", "categ_10", "categ_11", "categ_12", "categ_13", "categ_14", "categ_15", "categ_16", "categ_17", "categ_18", "categ_19", "categ_20", "categ_21", "categ_22", "categ_23", "categ_24", "categ_25", "categ_26"]


In [108]:
# colList=[]
# colList=labelColList+intColList+categColList
# trainDF = f_covert_to_df(trainRDD,colList)
# trainDF.cache()

In [109]:
# colList=[]
# colList=intColList+categColList
# testDF = f_covert_to_df(testRDD,colList)
# testDF.cache()

In [110]:
# trainDF = f_cast_str_to_int(trainDF,intColList)
# trainDF.cache()

In [111]:
# testDF = f_cast_str_to_int(testDF,intColList)
# testDF.cache()

##  * *Pending* * Toy Example

## Replicating databricks

In [ ]:
#http://lncohn.com/spark/ctr.html

### (1d) Define a OHE function


In [52]:
def one_hot_encoding(raw_feats, ohe_dict_broadcast, num_ohe_feats):
    """Produce a one-hot-encoding from a list of features and an OHE dictionary.

    Note:
        You should ensure that the indices used to create a SparseVector are sorted, and that the
        function handles missing features.

    Args:
        raw_feats (list of (int, str)): The features corresponding to a single observation.  Each
            feature consists of a tuple of featureID and the feature's value. (e.g. sample_one)
        ohe_dict_broadcast (Broadcast of dict): Broadcast variable containing a dict that maps
            (featureID, value) to unique integer.
        num_ohe_feats (int): The total number of unique OHE features (combinations of featureID and
            value).

    Returns:
        SparseVector: A SparseVector of length num_ohe_feats with indices equal to the unique
            identifiers for the (featureID, value) combinations that occur in the observation and
            with values equal to 1.0.
    """
    indices = sorted([ohe_dict_broadcast.value[feat] for feat in raw_feats if feat in ohe_dict_broadcast.value])
    values = np.ones(len(raw_feats))
    return SparseVector(num_ohe_feats,indices,values)


### (1e) Apply OHE to a dataset


In [72]:

def ohe_udf_generator(ohe_dict_broadcast):
    """Generate a UDF that is setup to one-hot-encode rows with the given dictionary.

    Note:
        We'll reuse this function to generate a UDF that can one-hot-encode rows based on a
        one-hot-encoding dictionary built from the training data.  Also, you should calculate
        the number of features before calling the one_hot_encoding function.

    Args:
        ohe_dict_broadcast (Broadcast of dict): Broadcast variable containing a dict that maps
            (featureID, value) to unique integer.

    Returns:
        UserDefinedFunction: A UDF can be used in `DataFrame` `select` statement to call a
            function on each row in a given column.  This UDF should call the one_hot_encoding
            function with the appropriate parameters.
    """
    length = len(ohe_dict_broadcast.value)
    return udf(lambda x: one_hot_encoding(x, ohe_dict_broadcast, length), VectorUDT())



### (2c) Automated creation of an OHE dictionary


In [39]:
def create_one_hot_dict(input_df):
    """Creates a one-hot-encoder dictionary based on the input data.

    Args:
        input_df (DataFrame with 'features' column): A DataFrame where each row contains a list of
            (featureID, value) tuples.

    Returns:
        dict: A dictionary where the keys are (featureID, value) tuples and map to values that are
            unique integers.
    """
    input_distinct_feats_df = input_df.select(explode(input_df.features)).distinct()
    input_ohe_dict = (input_distinct_feats_df
                     .rdd
                     .map(lambda r: tuple(r[0]))
                     .zipWithIndex().collectAsMap())
    return input_ohe_dict



### (3b) Extract features


In [55]:
# TODO: Replace <FILL IN> with the appropriate code

def parse_point(point):
    """Converts a comma separated string into a list of (featureID, value) tuples.

    Note:
        featureIDs should start at 0 and increase to the number of features - 1.

    Args:
        point (str): A comma separated string where the first value is the label and the rest
            are features.

    Returns:
        list: A list of (featureID, value) tuples.
    """
    values = point.split('\t')[1:]
    #values = filter(None, values)
    indices = range(len(values))
    return zip(indices,values)




### (3c) Extracting features continued

In [ ]:
parse_point_udf = udf(parse_point, ArrayType(StructType([StructField('_1', LongType()),StructField('_2', StringType())])))

def parse_raw_df(raw_df):
    """Convert a DataFrame consisting of rows of comma separated text into labels and feature.


    Args:
        raw_df (DataFrame with a 'text' column): DataFrame containing the raw comma separated data.

    Returns:
        DataFrame: A DataFrame with 'label' and 'feature' columns.   
  
    """
    return (raw_df.select(split(raw_df.text,'\t').getItem(0).cast("double").alias('label'),
                         parse_point_udf(raw_df.text).alias('features'))
                        .cache())


In [40]:
# sc.textFile('gs://bucket-w261-final/data/train.txt',50)
rawTrainDF = sqlContext.read.text('gs://bucket-w261-final/data/train.txt').withColumnRenamed("value", "text")
rawTestDF = sqlContext.read.text('gs://bucket-w261-final/data/test.txt').withColumnRenamed("value", "text")


In [41]:
rawTrainDF.cache()
rawTestDF.cache()

DataFrame[text: string]

In [96]:
parsed_train_df = parse_raw_df(rawTrainDF)


In [97]:
# 50% sampling due to memory constraint
parsed_train_df = parsed_train_df.sample(withReplacement=False, fraction=0.2, seed=1)

In [98]:
parsed_train_df.cache()

DataFrame[label: double, features: array<struct<_1:bigint,_2:string>>]

In [99]:
parsed_train_df.count()

9168574

In [100]:
num_categories = (parsed_train_df
                    .select(explode('features').alias('features'))
                    .distinct()
                    .select(col('features').getField('_1').alias('featureNumber'))
                    .groupBy('featureNumber')
                    .sum()
                    .orderBy('featureNumber')
                    .collect())


In [16]:
# !mkdir -p ./data
# !hadoop fs -get -f gs://bucket-w261-final/data/num_categories.txt data/num_categories.txt
# num_categories = pickle.load( open( "data/num_categories.txt", "rb" ) )


18/12/10 04:48:13 INFO gcs.GoogleHadoopFileSystemBase: GHFS version: hadoop2-1.9.7
18/12/10 04:48:14 WARN gcsio.GoogleCloudStorageReadChannel: Channel for 'gs://bucket-w261-final/data/num_categories.txt' is not open.


In [22]:
# import pickle
# !mkdir -p ./data
# with open('data/num_categories.txt', 'wb') as fp:
#     pickle.dump(num_categories, fp)


In [25]:
#!hadoop fs -put data/num_categories.txt gs://bucket-w261-final/data/num_categories.txt

18/12/10 01:43:25 INFO gcs.GoogleHadoopFileSystemBase: GHFS version: hadoop2-1.9.7


In [44]:
# !mkdir -p ./data
# !hadoop fs -get -f gs://bucket-w261-final/data/ctr_ohe_dict.pkl data/ctr_ohe_dict.pkl
# ctr_ohe_dict = pickle.load( open( "data/ctr_ohe_dict.pkl", "rb" ) )


18/12/10 06:33:16 INFO gcs.GoogleHadoopFileSystemBase: GHFS version: hadoop2-1.9.7
18/12/10 06:33:23 WARN gcsio.GoogleCloudStorageReadChannel: Channel for 'gs://bucket-w261-final/data/ctr_ohe_dict.pkl' is not open.


### (3d) Create an OHE dictionary from the dataset


In [101]:
ctr_ohe_dict = create_one_hot_dict(parsed_train_df)
num_ctr_ohe_feats = len(ctr_ohe_dict)
print(num_ctr_ohe_feats)
print(ctr_ohe_dict[(0, '')])


9454391
8509072


In [42]:
#f_save_to_bucket(ctr_ohe_dict,"ctr_ohe_dict")

18/12/10 02:00:21 INFO gcs.GoogleHadoopFileSystemBase: GHFS version: hadoop2-1.9.7
put: `gs://bucket-w261-final/data/ctr_ohe_dict.pkl': File exists
18/12/10 02:00:23 INFO gcs.GoogleHadoopFileSystemBase: GHFS version: hadoop2-1.9.7
-rwx------   3 root root 1061743533 2018-12-10 01:57 gs://bucket-w261-final/data/ctr_ohe_dict.pkl


### (3e) Apply OHE to the dataset


In [102]:
ohe_dict_broadcast = sc.broadcast(ctr_ohe_dict)
ohe_dict_udf =  ohe_udf_generator(ohe_dict_broadcast)


In [103]:
ohe_train_df =  parsed_train_df.select(parsed_train_df.label, ohe_dict_udf(parsed_train_df.features).alias('features'))
ohe_train_df.cache()


DataFrame[label: double, features: vector]

### * *PENDING* * Visualization 1: Feature frequency

### (3f) Handling unseen features


In [75]:
def one_hot_encoding_v2(raw_feats, ohe_dict_broadcast, num_ohe_feats):
    """Produce a one-hot-encoding from a list of features and an OHE dictionary.

    Note:
        You should ensure that the indices used to create a SparseVector are sorted, and that the
        function handles missing features.

    Args:
        raw_feats (list of (int, str)): The features corresponding to a single observation.  Each
            feature consists of a tuple of featureID and the feature's value. (e.g. sample_one)
        ohe_dict_broadcast (Broadcast of dict): Broadcast variable containing a dict that maps
            (featureID, value) to unique integer.
        num_ohe_feats (int): The total number of unique OHE features (combinations of featureID and
            value).

    Returns:
        SparseVector: A SparseVector of length num_ohe_feats with indices equal to the unique
            identifiers for the (featureID, value) combinations that occur in the observation and
            with values equal to 1.0.
    """
    indices = sorted([ohe_dict_broadcast.value[feat] for feat in raw_feats if feat in ohe_dict_broadcast.value])
    values = np.ones(len(raw_feats))
    return SparseVector(num_ohe_feats,indices,values)

def ohe_udf_generator_v2(ohe_dict_broadcast):
    """Generate a UDF that is setup to one-hot-encode rows with the given dictionary.

    Note:
        We'll reuse this function to generate a UDF that can one-hot-encode rows based on a
        one-hot-encoding dictionary built from the training data.  Also, you should calculate
        the number of features before calling the one_hot_encoding function.

    Args:
        ohe_dict_broadcast (Broadcast of dict): Broadcast variable containing a dict that maps
            (featureID, value) to unique integer.

    Returns:
        UserDefinedFunction: A UDF can be used in `DataFrame` `select` statement to call a
            function on each row in a given column.  This UDF should call the one_hot_encoding
            function with the appropriate parameters.
    """
    length = len(ohe_dict_broadcast.value)
    return udf(lambda x: one_hot_encoding_v2(x, ohe_dict_broadcast, length), VectorUDT())


In [104]:
ohe_dict_missing_udf = ohe_udf_generator_v2(ohe_dict_broadcast)


## Part 4: CTR prediction and logloss evaluation


### (4a) Logistic regression


In [105]:
standardization = False
elastic_net_param = 0.0
reg_param = .01
max_iter = 20

lr = LogisticRegression(featuresCol="features", labelCol="label", predictionCol="prediction", maxIter=max_iter, regParam=reg_param, elasticNetParam=elastic_net_param, fitIntercept=True,  standardization=standardization)

lr_model_basic = lr.fit(ohe_train_df)

print('intercept: {0}'.format(lr_model_basic.intercept))
print('length of coefficients: {0}'.format(len(lr_model_basic.coefficients)))
sorted_coefficients = sorted(lr_model_basic.coefficients)[:5]
print(sorted_coefficients)


Py4JJavaError: An error occurred while calling o1116.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 39.0 failed 4 times, most recent failure: Lost task 8.3 in stage 39.0 (TID 3532, cluster-w261-w-3.c.w266-203603.internal, executor 90): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 230, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 225, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 324, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 139, in dump_stream
    for obj in iterator:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 313, in _batched
    for item in 

### (4b) Log loss


In [138]:
# from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# # Create ParamGrid for Cross Validation
# paramGrid = (ParamGridBuilder()
#              .addGrid(lr.regParam, [0.01, 0.5, 2.0])
#              .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
#              .addGrid(lr.maxIter, [1, 5, 10])
#              .build())


In [ ]:
# # Create 5-fold CrossValidator
# cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# # Run cross validations
# cvModel = cv.fit(trainSubsetOHEdf)
# # this will likely take a fair amount of time because of the amount of models that we're creating and testing


In [ ]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
# evaluator.evaluate(predictions)
